# Network component analysis

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import networkx as nx
from networkx.algorithms import community 

In [2]:
# read all csv files from edgelists folder and save to edgelists_df
edgelists_df = pd.concat([pd.read_csv(file) for file in Path("edgelists").iterdir()], ignore_index=True)
# drop duplicates in edgelists_df
edgelists_df.drop_duplicates(inplace=True)
print(edgelists_df)
edgelists_graph_sample = edgelists_df['user_id'].unique()
author_ids_sample = np.random.choice(edgelists_graph_sample, 200, replace=False)
edgelists_df_sample = edgelists_df[edgelists_df['user_id'].isin(author_ids_sample)]

G = nx.from_pandas_edgelist(edgelists_df_sample, source='user_id', target='author_id', create_using=nx.Graph())
# print number of nodes in edgelists_graph_sample
print(f"{len(G.nodes)=}")

# print number of edges in edgelists_graph_sample
print(f"{len(G.edges)=}")

                     user_id  author_id
0                  159648236  454278875
1                  272581313  454278875
2                  400860401  454278875
3                 1103560652  454278875
4                   43412258  454278875
...                      ...        ...
5033373            325674737  554497503
5033376           4191445095  554497503
5033377            878977598  554497503
5033378            481769384  554497503
5033380  1462535282172084225  554497503

[4466776 rows x 2 columns]
len(G.nodes)=261
len(G.edges)=328


In [45]:
louvain_community = community.louvain_communities(G)
# louvain_first = next(louvain_community)
print(len(louvain_community))

ZeroDivisionError: division by zero

In [17]:
# Louvain implementation
def louvain(G, npassage):
    # Will contain the graph and the communities after each passage
    data = {}
    for i in range(npassage):
        G, communities = louvain_step(G)
        data[i] = (G, communities)
        print(f"There are {len(communities)} communities after passage {i}")
    return data


def louvain_step(G):
    # Communities are stored in a dict
    # Step 1: Initialization, start with each node being a single community
    communities = {idx:[node] for idx, node in enumerate(G.nodes)}
    
    # TODO: ATM it is a fixed number of iteration, we need to find a way to
    # Stop the iterations whenever the communities are not evolving anymore
    for i in range(50):
        print('.', flush=True, end='')
        for node in G.nodes:
            # Step 2: Remove node from its community
            belong_to = get_community(node, communities)
            communities[belong_to].remove(node)
            if communities[belong_to] == []:
                del communities[belong_to]

            # Step 3: Insert the node in the community that maximizes the modularity
            neighboring_communities = get_neighboring_communities(G, node, communities)
            scores = [(neighbor_community, modularity(G, node, communities[neighbor_community])) for neighbor_community in neighboring_communities]
            best_comm, best_score = max(scores, key=lambda x: x[1])
            communities[best_comm].append(node)
            
#     return get_new_graph(G, communities), communities
    return G, communities

                
def get_community(node, communities):
    for idx, community in communities.items():
        if node in community:
            return idx
    assert False, f"Node {node} not found"
    
    
def get_neighboring_communities(G, node, communities):
    neighboring_communities = []
    for idx, community in communities.items():
        for member in community:
            # G[node] returns the neighbors of node
            if member in [neighbor for neighbor in G[node]]:
                neighboring_communities.append(idx)
                continue
    assert neighboring_communities != [], f"No neighboring communities for node {node}"
    return neighboring_communities
    

def modularity(G, node, community):
    shared_links = 0
    deg_sum_community = 0
    for member in community:
        deg_sum_community += G.degree[member]
        if member in [neighbor for neighbor in G[node]]:
            # Shared links are counted in both direction: a -> b and b -> a
            shared_links += 2
    
    return (1 / (2 * len(G.edges))) * (shared_links - (deg_sum_community*G.degree[node]/len(G.edges)))


# def get_new_graph(old_graph, communities):
# TODO: construct this method, IDK how to use the weighted edges in the modularity function/calculation
    
    
louvain(G, 3)

..................................................There are 55 communities after passage 0
..................................................There are 55 communities after passage 1
..................................................There are 55 communities after passage 2


{0: (<networkx.classes.graph.Graph at 0x7f46e851b4f0>,
  {1: [2764805061,
    1941753763,
    2327862405,
    2432700673,
    85597233,
    1311005371448795137,
    292211300,
    1077405188443566087,
    125570156,
    708010372687462401,
    745783114866143232,
    1434292939,
    1224305118,
    362742083,
    221079261,
    803335706,
    75050446,
    456643729,
    1020357617422544897,
    3067223383,
    225832762,
    1475533674783379457,
    1309663611384725504,
    1352336498700419072,
    1487307690905042944,
    1451987518405054466,
    28852017,
    1646067066,
    2347592418,
    913264160087072769,
    576287528],
   5: [4845456638, 1403364703284436992, 302246016],
   7: [1496087699329015808, 4448232618, 713717831423107072],
   12: [15400294,
    477153872,
    862757368353796096,
    130397535,
    246381716,
    1259561624,
    1160600400464158720],
   18: [23170408, 1307865457903448064],
   20: [568110066, 1497194071105933312, 253432854],
   22: [1498149788151427078, 

## TODO:
- Sample by taking random author_id and take all edges connected to it
- Implement from scratch community detection: Louvain (faster than Girvan)
- Look at how to store the communities (edgelist or so)
- Look at the caracteristics of the huge communities: what is the Tweet, who is the author, what are the hashtags etc...
- Make profiles of users: press, random guy, famous guy, etc...
- Look at outliers
- Load communities to Gephi to vizualise them